# SAL Script - Slew and Track

Performs a single slew and track using SAL Script.  
It can be used as a minimum functionality prototype for the Soak Test.

## Notebook Setup

Start by importing relevat libraries, setting up domain, remotes, logs, etc.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import asyncio
import logging
import yaml


import numpy as np

from astropy import units as u
from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.standardscripts.maintel.track_target import TrackTarget
from lsst.sitcom import vandv

In [ ]:
track_target = TrackTarget(index=199)
await track_target.start_task

In [ ]:
mtcs = track_target.tcs

In [ ]:
vandv.logger.add_filter_to_mtcs()

## Prepare Components

In [ ]:
await mtcs.set_state(
    salobj.State.DISABLED, 
    components=[
        "mtmount", 
        "mtrotator",
        "mtptg",
    ]
)


In [ ]:
await mtcs.set_state(
    salobj.State.ENABLED, 
    components=[
        "mtmount",
        # "mtrotator", 
        "mtptg",
    ]
)

await mtcs.rem.mtmount.cmd_setLogLevel.set_start(level=15)

# await mtcs.disable_ccw_following()

In [ ]:
az = mtcs.rem.mtmount.tel_azimuth.get().actualPosition
el = mtcs.rem.mtmount.tel_elevation.get().actualPosition

print(az, el)

In [ ]:
await mtcs.rem.mtmount.cmd_homeBothAxes.start(timeout=300)

## Helper Functions

In [ ]:
async def slew_and_track(az, el, track_for, rot_type="Physical", rot_value=0):
    """Use the `TrackTarget` standard script to slew and track 
    one coordinate emulating the Script Queue.
    
    Parameters
    ----------
    az : `float`
        Azimuth in hour angle.
    el :  `float`
        Elevation in degrees.
    track_for : `float` 
        Number of seconds to track for.
    rot_type : `str`
        Type of rotation tracking. 
    rot_value : `float` 
        Rotation angle in degrees.
    """
    radec = track_target.tcs.radec_from_azel(az, el)
    
    configuration = yaml.safe_dump(
        {
            "slew_icrs": {
                "ra": float(radec.ra.hour),
                "dec": float(radec.dec.deg),
            }, 
            "rot_value": rot_value, 
            "rot_type": rot_type, 
            "track_for": track_for, 
            "stop_when_done": False,
            "ignore": [
                "mtaos",
                "mtdome",
                "mtdometrajectory",
                "mthexapod_1",
                "mthexapod_2", 
                "mtm1m3", 
                "mtm2",
                "mtrotator",
            ],
        }
    )

    # Set script state to UNCONFIGURED
    # this is required to run the script a 2nd time but otherwise is a no-op
    await track_target.set_state(ScriptState.UNCONFIGURED)

    # Configure the script, which puts the ScriptState to CONFIGURED
    config_data = track_target.cmd_configure.DataType()
    config_data.config = configuration

    await track_target.do_configure(config_data)
    results = await track_target.run()

## Run Observation Simulation

In [ ]:
az = mtcs.rem.mtmount.tel_azimuth.get().actualPosition
el = mtcs.rem.mtmount.tel_elevation.get().actualPosition

print(az, el)

In [ ]:
azel_generator = vandv.slew_and_track.random_walk_azel_by_time(
    total_time=0.1 * 3600.,
    mtmount=mtcs.rem.mtmount,
    logger=track_target.log, 
    big_slew_prob=0.10, # 10% probability
    big_slew_radius=9.0,
)

for az, el in azel_generator:

    ## Comment the line below if you want to run with hardware
    # await asyncio.sleep(1)
    
    ## Uncomment the line below if you want to run with hardware
    await slew_and_track(az, el, 39, rot_type="Physical", rot_value=0)

In [ ]:
az_grid = [180, 135, 90, 45, 0, -45, -90]
el_grid = [30, 60, 85]

azel_generator = vandv.slew_and_track.azel_grid_by_time(
    total_time=60, 
    _az_grid=az_grid, 
    _el_grid=el_grid,
    logger=track_target.log,
)

for az, el in azel_generator:
    
    ## Comment the line below if you want to run with hardware
    await asyncio.sleep(1)
    
    ## Uncomment the line below if you want to run with hardware
    # await slew_and_track(az, el, 39, rot_type="Physical", rot_value=0)

In [ ]:
await mtcs.point_azel(az=0, el=80)
await mtcs.stop_tracking()